# Heterogeneous flow

The best way to run `foxes` calculations on heterogeneous background flow fields is by providing them in `netCDF` format. They should contain the following coordinates:

- A state coordinate, e.g. `Time` (expected by default) or `state`, or similar
- A height coordinate, e.g. `height` (expected by default) or `h`, or similar
- A `y` coordinate, e.g. `UTMY` (expected by default) or `y`, or similar
- A `x` coordinate, e.g. `UTMX` (expected by default) or `x`, or similar

The file may contain any kind of `foxes` variables as data fields, e.g.:

- Wind speed data, e.g. `WS` (expected by default, if claimed as output variable), `ws` or similar
- Wind direction data, e.g. `WD` (expected by default, if claimed as output variable), `wd` or similar
- Turbulence intensity data, e.g. `TI` (expected by default, if claimed as output variable), `ti` or similar
- Air density data, e.g. `RHO` (expected by default, if claimed as output variable), `rho` or similar

All data must depend on the state coordinate, and may depend on the others.

These are the required imports for this example:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import foxes
import foxes.variables as FV
import foxes.constants as FC

One very simple example for `netCDF` type data is provided in the static data, under the name `wind_rotation.nc`. It contains two states, two heights, and simple 2 x 2 horizontal data that describes identical wind speeds at all four corner points associated with different wind direction values. It can be loaded as follows:

In [ ]:
states = foxes.input.states.FieldDataNC(
    data_source="wind_rotation.nc",
    states_coord="state",
    x_coord="x",
    y_coord="y",
    h_coord="h",
    time_format=None,
    output_vars=[FV.WS, FV.WD, FV.TI, FV.RHO],
    var2ncvar={FV.WS: "ws", FV.WD: "wd"},
    fixed_vars={FV.RHO: 1.225, FV.TI: 0.1},
    pre_load=True,
    bounds_error=False,
)

Note that it is recommended that the `states` object should be created outside the `DaskRunner` when working with NetCFD input.

Now back to our example. Let's place a simple 3 x 3 grid wind farm inside the data domain, which is a rectangle between `(0, 0)` and `(2500, 2500)`:

In [ ]:
farm = foxes.WindFarm()
foxes.input.farm_layout.add_grid(
    farm,
    xy_base=np.array([500.0, 500.0]),
    step_vectors=np.array([[500.0, 0], [0, 500.0]]),
    steps=(3, 3),
    turbine_models=["NREL5MW"],
    verbosity=0,
)

The streamline following wakes are realized by selecting a `wake frame` that is an instance of `foxes.models.wake_frames.Streamlines`, e.g. the model `streamlines_100` in the model book. This model has a streamline step size of 100 m:

In [ ]:
mbook = foxes.ModelBook()

algo = foxes.algorithms.Downwind(
    mbook,
    farm,
    states=states,
    rotor_model="grid16",
    wake_models=["Jensen_linear_k007"],
    wake_frame="streamlines_100",
    partial_wakes_model="auto",
    chunks={FC.STATE: 1000, FC.POINT: 4000},
    verbosity=0,
)

We run the algorithm, once explicitely for calculating the wind farm data, and once implicitely when creating horizontal flow plots:

In [ ]:
farm_results = algo.calc_farm()

fr = farm_results.to_dataframe()
print(fr[[FV.WD, FV.AMB_REWS, FV.REWS, FV.AMB_P, FV.P]])

o = foxes.output.FlowPlots2D(algo, farm_results)
for fig in o.gen_states_fig_xy(
    FV.WS,
    resolution=10,
    figsize=(8, 8),
    quiver_pars=dict(angles="xy", scale_units="xy", scale=0.07),
    quiver_n=15,
    xspace=1000,
    yspace=1000,
):
    plt.show()
    plt.close(fig)